* EDA3에서 저장한 info데이터를 가져온다.
* 리스트 형태로 되어있는 칼럼을 풀어서 하나의 df로 합쳐준다.
    * 풀어줄 칼럼은 team 컬럼과 participants 컬럼
* 최종적으로 각 행은 하나의 match 데이터를 가진 통합데이터를 만들고 저장한다.

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from tqdm import tnrange
import time
from sklearn.model_selection import train_test_split
from pandas.io.json import json_normalize

In [2]:
api_key = 'RGAPI-eabfcbf1-25a3-4971-be6c-036dadfe7f42'

In [3]:
data = pd.read_pickle('data/saving_data/match_info.pkl')
data

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589372097554,1712,4377039377,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588139841997,1369,4339975651,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589425548230,1508,4378064003,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589276891396,1325,4374104072,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1587381375511,1055,4317268687,CLASSIC,MATCHED_GAME,10.8.317.8137,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."


In [4]:
# 모든값이 MaM인 status칼럼 삭제
data = data.drop(['status'], axis=1)
data.head()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."


In [5]:
'''
NaN이 들어있는 행은 삭제시켜 teams와 participants칼럼을 풀어서 하나의 dataframe으로 합칠 수 있게 만든다.
'''
data = data.dropna()

In [6]:
data.isnull().sum().sum()

0

In [7]:
data

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589372097554,1712,4377039377,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588139841997,1369,4339975651,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589425548230,1508,4378064003,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589276891396,1325,4374104072,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1587381375511,1055,4317268687,CLASSIC,MATCHED_GAME,10.8.317.8137,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."


In [8]:
'''
teams 칼럼에 들어있는 데이터를 풀어서 하나의 칼럼을 갖도록 df를 만든다.
두개의 팀이 있으므로 각 팀을 나눠서 df를 만들고 합쳐준다.
100 --> blue side
200 --> red side
'''
# blue side
a_ls = list(data['teams'])

team1_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][0].pop('bans',None)
        team1 = pd.DataFrame(list(a_ls[i][0].values()),index = list(a_ls[i][0].keys())).T
        team1_df = team1_df.append(team1)
    except:
        print(i)
    
team1_df.index = range(len(team1_df))

In [9]:
'''
칼럼의 이름이 blue와 red팀이 같아질수 있으므로 blue팀은 칼럼이름앞에 blue 라고 추가해준다.
'''
blue_columns = ['blue'+s for s in list(team1_df.columns)]
blue_columns

['blueteamId',
 'bluewin',
 'bluefirstBlood',
 'bluefirstTower',
 'bluefirstInhibitor',
 'bluefirstBaron',
 'bluefirstDragon',
 'bluefirstRiftHerald',
 'bluetowerKills',
 'blueinhibitorKills',
 'bluebaronKills',
 'bluedragonKills',
 'bluevilemawKills',
 'blueriftHeraldKills',
 'bluedominionVictoryScore']

In [10]:
team1_df.head()

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore
0,100,Win,True,False,True,True,True,False,8,1,1,1,0,1,0
1,100,Fail,False,False,False,False,False,False,1,0,0,2,0,0,0
2,100,Fail,True,True,False,False,True,False,3,0,0,3,0,0,0
3,100,Fail,False,False,False,False,True,False,0,0,0,1,0,0,0
4,100,Win,True,True,True,False,True,True,7,1,0,1,0,1,0


In [11]:
'''
칼럼이름이 바뀐것을 확인
'''
team1_df.columns = blue_columns
team1_df

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,bluebaronKills,bluedragonKills,bluevilemawKills,blueriftHeraldKills,bluedominionVictoryScore
0,100,Win,True,False,True,True,True,False,8,1,1,1,0,1,0
1,100,Fail,False,False,False,False,False,False,1,0,0,2,0,0,0
2,100,Fail,True,True,False,False,True,False,3,0,0,3,0,0,0
3,100,Fail,False,False,False,False,True,False,0,0,0,1,0,0,0
4,100,Win,True,True,True,False,True,True,7,1,0,1,0,1,0
5,100,Fail,True,True,False,False,True,True,4,0,0,1,0,2,0
6,100,Fail,False,False,False,False,False,False,2,0,0,0,0,0,0
7,100,Fail,True,False,False,False,False,False,1,0,0,1,0,0,0
8,100,Win,True,True,False,True,True,True,6,0,1,2,0,2,0
9,100,Fail,False,True,False,False,False,True,1,0,0,0,0,1,0


In [12]:
'''
red team
'''
team2_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][1].pop('bans',None)
        team2 = pd.DataFrame(list(a_ls[i][1].values()),index = list(a_ls[i][1].keys())).T
        team2_df = team2_df.append(team2)
    except:
        print(i)
    
team2_df.index = range(len(team2_df))

In [13]:
team2_df.head()

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore
0,200,Fail,False,True,False,False,False,True,3,0,0,2,0,1,0
1,200,Win,True,True,True,True,True,True,7,1,1,1,0,1,0
2,200,Win,False,False,True,True,False,True,9,3,1,1,0,2,0
3,200,Win,True,True,True,False,False,True,11,1,0,0,0,1,0
4,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0


In [14]:
red_columns = ['red'+s for s in list(team2_df.columns)]
red_columns

['redteamId',
 'redwin',
 'redfirstBlood',
 'redfirstTower',
 'redfirstInhibitor',
 'redfirstBaron',
 'redfirstDragon',
 'redfirstRiftHerald',
 'redtowerKills',
 'redinhibitorKills',
 'redbaronKills',
 'reddragonKills',
 'redvilemawKills',
 'redriftHeraldKills',
 'reddominionVictoryScore']

In [15]:
'''
칼럼이름이 바뀐것을 확인
'''
team2_df.columns = red_columns
team2_df

,redteamId,redwin,redfirstBlood,redfirstTower,redfirstInhibitor,redfirstBaron,redfirstDragon,redfirstRiftHerald,redtowerKills,redinhibitorKills,redbaronKills,reddragonKills,redvilemawKills,redriftHeraldKills,reddominionVictoryScore
0,200,Fail,False,True,False,False,False,True,3,0,0,2,0,1,0
1,200,Win,True,True,True,True,True,True,7,1,1,1,0,1,0
2,200,Win,False,False,True,True,False,True,9,3,1,1,0,2,0
3,200,Win,True,True,True,False,False,True,11,1,0,0,0,1,0
4,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0
5,200,Win,False,False,True,True,False,False,9,2,1,3,0,0,0
6,200,Win,True,True,True,True,True,True,9,1,1,3,0,1,0
7,200,Win,False,True,True,True,True,True,9,2,1,3,0,2,0
8,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0
9,200,Win,True,False,True,False,True,False,7,1,0,2,0,1,0


In [17]:
'''
각 팀의 info를 담은 dataframe의 길이가 일치하는지 확인해준다.
'''
len(team1_df) == len(team2_df) == len(data)

True

* data에서 teams, participants 그리고 duration 칼럼을 사용할것
* [column info](https://developer.riotgames.com/apis#match-v4/GET_getMatch) 에서 각 내용 확인 가능
* 각팀으로 만든 df와 기존 data를 합쳐준다

In [18]:
'''
팀1과 팀2를 합쳐준 df에 추가로 합쳐줄 gameDuration df 생성
--> 이때 reset_index를 꼭 해줘야한다.
    ---> 안하면 cannot reindex from a duplicate axis 오류남
'''
data_pri = data[['gameDuration']]
data_pri = data_pri.reset_index()
data_pri.drop(['index'],axis=1, inplace=True)
data_pri

,gameDuration
0,1475
1,1404
2,1722
3,965
4,812
5,1712
6,1369
7,1508
8,1325
9,1055


In [19]:
'''
team df와 기존 data에서 사용할 칼럼을 선택해 합쳐준다.
먼저 팀1과 팀2를 합쳐주고
'''
data_team = pd.concat([team1_df,team2_df,data_pri],axis=1)

In [20]:
data_team

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,redfirstDragon,redfirstRiftHerald,redtowerKills,redinhibitorKills,redbaronKills,reddragonKills,redvilemawKills,redriftHeraldKills,reddominionVictoryScore,gameDuration
0,100,Win,True,False,True,True,True,False,8,1,...,False,True,3,0,0,2,0,1,0,1475
1,100,Fail,False,False,False,False,False,False,1,0,...,True,True,7,1,1,1,0,1,0,1404
2,100,Fail,True,True,False,False,True,False,3,0,...,False,True,9,3,1,1,0,2,0,1722
3,100,Fail,False,False,False,False,True,False,0,0,...,False,True,11,1,0,0,0,1,0,965
4,100,Win,True,True,True,False,True,True,7,1,...,False,False,0,0,0,0,0,0,0,812
5,100,Fail,True,True,False,False,True,True,4,0,...,False,False,9,2,1,3,0,0,0,1712
6,100,Fail,False,False,False,False,False,False,2,0,...,True,True,9,1,1,3,0,1,0,1369
7,100,Fail,True,False,False,False,False,False,1,0,...,True,True,9,2,1,3,0,2,0,1508
8,100,Win,True,True,False,True,True,True,6,0,...,False,False,0,0,0,0,0,0,0,1325
9,100,Fail,False,True,False,False,False,True,1,0,...,True,False,7,1,0,2,0,1,0,1055


In [21]:
'''
최종적으로 통합데이터 NaN값 확인 --> 0 확인 완료
'''
data_team.isnull().sum().sum()

0

* team 칼럼을 풀어서 하나의 행으로 만드는 과정이 끝났다.
* 이와 같은 과정을 participants 칼럼에도 적용해야 한다.
* 다만, participants 칼럼은 각 플레이어의 정보를 담고있으므로 총 10명의 정보가 등장하고 많은 정보를 담고있으므로 추후 칼럼을 선별해야할것

In [22]:
p_ls = list(data['participants'])

In [81]:
df_party = pd.DataFrame()
df_party

""


In [78]:
'''
participants 칼럼에서 실제로 사용할 칼럼들의 목록
stats. timeline. 과 같이 . 으로 구분되는 이유는 json_normalize를 사용하기때문이다.
remain_col_ver1 은 20분까지의 경기 데이터가 있을 경우이며
remain_col_ver2 는 20분전에 경기가 끝나서 0-10의 데이터만 있는 경우이다.
remain_col_ver3 는 10분전에 경기가 끝난경우 --> 미드오픈같은 경우가 있다.
remain_col_ver1 리스트만 가지고 아래 for문을 실행하면 20분전에 경기가 끝난 경우 에러가 발생하기 떄문에
세 가지의 case로 나눠서 처리한다.

'''
remain_col_ver1 = ['participantId','teamId','championId','stats.win','stats.kills','stats.deaths',
              'stats.assists','stats.longestTimeSpentLiving','stats.visionScore','stats.goldEarned',
              'stats.totalMinionsKilled','stats.champLevel','stats.visionWardsBoughtInGame',
              'timeline.lane',
              'timeline.creepsPerMinDeltas.0-10','timeline.creepsPerMinDeltas.10-20',
              'timeline.xpPerMinDeltas.0-10','timeline.xpPerMinDeltas.10-20',
              'timeline.goldPerMinDeltas.0-10','timeline.goldPerMinDeltas.10-20']

remain_col_ver2 = ['participantId','teamId','championId','stats.win','stats.kills','stats.deaths',
              'stats.assists','stats.longestTimeSpentLiving','stats.visionScore','stats.goldEarned',
              'stats.totalMinionsKilled','stats.champLevel','stats.visionWardsBoughtInGame',
              'timeline.lane',
              'timeline.creepsPerMinDeltas.0-10',
              'timeline.xpPerMinDeltas.0-10',
              'timeline.goldPerMinDeltas.0-10']

remain_col_ver3 = ['participantId','teamId','championId','stats.win','stats.kills','stats.deaths',
              'stats.assists','stats.longestTimeSpentLiving','stats.visionScore','stats.goldEarned',
              'stats.totalMinionsKilled','stats.champLevel','stats.visionWardsBoughtInGame',
              'timeline.lane']

print(len(remain_col))

20


In [82]:
df_party = pd.DataFrame()
for i in tnrange(len(p_ls), desc='1st Loop'):
    df_middle = pd.DataFrame()
    for j in range(10):
        a1 = json_normalize(p_ls[i][j])
        try:
            a1 = a1[remain_col_ver1]
        except:
            try:
                a1 = a1[remain_col_ver2]
            except:
                a1 = a1[remain_col_ver3]
        a1.columns = ['{}'.format(j+1) + s for s in a1.columns]
        df_middle = pd.concat([df_middle,a1],axis=1)
    df_party = df_party.append(df_middle, ignore_index=True)

C:\Users\zone\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


C:\Users\zone\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [83]:
df_party

,10championId,10participantId,10stats.assists,10stats.champLevel,10stats.deaths,10stats.goldEarned,10stats.kills,10stats.longestTimeSpentLiving,10stats.totalMinionsKilled,10stats.visionScore,...,9stats.visionWardsBoughtInGame,9stats.win,9teamId,9timeline.creepsPerMinDeltas.0-10,9timeline.creepsPerMinDeltas.10-20,9timeline.goldPerMinDeltas.0-10,9timeline.goldPerMinDeltas.10-20,9timeline.lane,9timeline.xpPerMinDeltas.0-10,9timeline.xpPerMinDeltas.10-20
0,21,10,7,12,12,8704,3,258,131,24,...,6,False,200,2.2,1.1,189.7,262.7,BOTTOM,280.7,305.2
1,350,10,18,13,1,8064,3,1162,4,35,...,5,True,200,8.6,9.9,369.2,834.7,BOTTOM,326.0,753.0
2,60,10,12,15,4,10973,4,708,22,66,...,4,True,200,6.6,7.2,296.8,524.6,JUNGLE,416.8,516.6
3,518,10,2,10,1,5923,1,451,106,13,...,5,True,200,0.5,NaN,414.5,NaN,NONE,424.5,NaN
4,43,10,0,6,1,2519,0,500,13,3,...,0,False,200,6.3,NaN,244.0,NaN,NONE,226.5,NaN
5,110,10,12,15,4,12661,7,676,206,42,...,12,True,200,1.1,1.2,221.1,312.2,BOTTOM,240.1,356.9
6,523,10,11,12,6,10343,8,389,101,25,...,6,True,200,5.8,11.3,318.7,637.0,MIDDLE,384.7,616.4
7,134,10,10,15,0,12850,10,0,201,22,...,6,True,200,0.6,1.4,308.7,518.8,JUNGLE,323.2,635.2
8,59,10,3,10,5,6484,2,354,18,18,...,5,False,200,1.3,1.4,195.6,221.2,BOTTOM,265.4,241.7
9,7,10,2,11,1,5865,1,663,125,20,...,1,True,200,1.2,NaN,148.3,NaN,NONE,277.1,NaN


In [84]:
df_info = pd.concat([data_team, df_party] ,axis=1)

In [93]:
df_info

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,9stats.visionWardsBoughtInGame,9stats.win,9teamId,9timeline.creepsPerMinDeltas.0-10,9timeline.creepsPerMinDeltas.10-20,9timeline.goldPerMinDeltas.0-10,9timeline.goldPerMinDeltas.10-20,9timeline.lane,9timeline.xpPerMinDeltas.0-10,9timeline.xpPerMinDeltas.10-20
0,100,Win,True,False,True,True,True,False,8,1,...,6,False,200,2.2,1.1,189.7,262.7,BOTTOM,280.7,305.2
1,100,Fail,False,False,False,False,False,False,1,0,...,5,True,200,8.6,9.9,369.2,834.7,BOTTOM,326.0,753.0
2,100,Fail,True,True,False,False,True,False,3,0,...,4,True,200,6.6,7.2,296.8,524.6,JUNGLE,416.8,516.6
3,100,Fail,False,False,False,False,True,False,0,0,...,5,True,200,0.5,NaN,414.5,NaN,NONE,424.5,NaN
4,100,Win,True,True,True,False,True,True,7,1,...,0,False,200,6.3,NaN,244.0,NaN,NONE,226.5,NaN
5,100,Fail,True,True,False,False,True,True,4,0,...,12,True,200,1.1,1.2,221.1,312.2,BOTTOM,240.1,356.9
6,100,Fail,False,False,False,False,False,False,2,0,...,6,True,200,5.8,11.3,318.7,637.0,MIDDLE,384.7,616.4
7,100,Fail,True,False,False,False,False,False,1,0,...,6,True,200,0.6,1.4,308.7,518.8,JUNGLE,323.2,635.2
8,100,Win,True,True,False,True,True,True,6,0,...,5,False,200,1.3,1.4,195.6,221.2,BOTTOM,265.4,241.7
9,100,Fail,False,True,False,False,False,True,1,0,...,1,True,200,1.2,NaN,148.3,NaN,NONE,277.1,NaN


In [99]:
'''
추가적으로 gameDuration 이 초로 나타나져있으므로 분으로 나타내주는 컬럼(gameDurationM)도 하나 만들어준다.
'''
df_info = df_info.astype({'gameDuration': 'int32'})
df_info['gameDurationM'] = df_info['gameDuration'].div(60).round(2)

In [100]:
df_info

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,9stats.win,9teamId,9timeline.creepsPerMinDeltas.0-10,9timeline.creepsPerMinDeltas.10-20,9timeline.goldPerMinDeltas.0-10,9timeline.goldPerMinDeltas.10-20,9timeline.lane,9timeline.xpPerMinDeltas.0-10,9timeline.xpPerMinDeltas.10-20,gameDurationM
0,100,Win,True,False,True,True,True,False,8,1,...,False,200,2.2,1.1,189.7,262.7,BOTTOM,280.7,305.2,24.58
1,100,Fail,False,False,False,False,False,False,1,0,...,True,200,8.6,9.9,369.2,834.7,BOTTOM,326.0,753.0,23.40
2,100,Fail,True,True,False,False,True,False,3,0,...,True,200,6.6,7.2,296.8,524.6,JUNGLE,416.8,516.6,28.70
3,100,Fail,False,False,False,False,True,False,0,0,...,True,200,0.5,NaN,414.5,NaN,NONE,424.5,NaN,16.08
4,100,Win,True,True,True,False,True,True,7,1,...,False,200,6.3,NaN,244.0,NaN,NONE,226.5,NaN,13.53
5,100,Fail,True,True,False,False,True,True,4,0,...,True,200,1.1,1.2,221.1,312.2,BOTTOM,240.1,356.9,28.53
6,100,Fail,False,False,False,False,False,False,2,0,...,True,200,5.8,11.3,318.7,637.0,MIDDLE,384.7,616.4,22.82
7,100,Fail,True,False,False,False,False,False,1,0,...,True,200,0.6,1.4,308.7,518.8,JUNGLE,323.2,635.2,25.13
8,100,Win,True,True,False,True,True,True,6,0,...,False,200,1.3,1.4,195.6,221.2,BOTTOM,265.4,241.7,22.08
9,100,Fail,False,True,False,False,False,True,1,0,...,True,200,1.2,NaN,148.3,NaN,NONE,277.1,NaN,17.58


In [101]:
'''
통합데이터 저장
'''
df_info.to_pickle('data/saving_data/df_info_EDA4')

In [102]:
'''
통합데이터 불러와서 출력 확인
'''
loading_test = pd.read_pickle('data/saving_data/df_info_EDA4')
loading_test

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,9stats.win,9teamId,9timeline.creepsPerMinDeltas.0-10,9timeline.creepsPerMinDeltas.10-20,9timeline.goldPerMinDeltas.0-10,9timeline.goldPerMinDeltas.10-20,9timeline.lane,9timeline.xpPerMinDeltas.0-10,9timeline.xpPerMinDeltas.10-20,gameDurationM
0,100,Win,True,False,True,True,True,False,8,1,...,False,200,2.2,1.1,189.7,262.7,BOTTOM,280.7,305.2,24.58
1,100,Fail,False,False,False,False,False,False,1,0,...,True,200,8.6,9.9,369.2,834.7,BOTTOM,326.0,753.0,23.40
2,100,Fail,True,True,False,False,True,False,3,0,...,True,200,6.6,7.2,296.8,524.6,JUNGLE,416.8,516.6,28.70
3,100,Fail,False,False,False,False,True,False,0,0,...,True,200,0.5,NaN,414.5,NaN,NONE,424.5,NaN,16.08
4,100,Win,True,True,True,False,True,True,7,1,...,False,200,6.3,NaN,244.0,NaN,NONE,226.5,NaN,13.53
5,100,Fail,True,True,False,False,True,True,4,0,...,True,200,1.1,1.2,221.1,312.2,BOTTOM,240.1,356.9,28.53
6,100,Fail,False,False,False,False,False,False,2,0,...,True,200,5.8,11.3,318.7,637.0,MIDDLE,384.7,616.4,22.82
7,100,Fail,True,False,False,False,False,False,1,0,...,True,200,0.6,1.4,308.7,518.8,JUNGLE,323.2,635.2,25.13
8,100,Win,True,True,False,True,True,True,6,0,...,False,200,1.3,1.4,195.6,221.2,BOTTOM,265.4,241.7,22.08
9,100,Fail,False,True,False,False,False,True,1,0,...,True,200,1.2,NaN,148.3,NaN,NONE,277.1,NaN,17.58


In [103]:
s = ['1' + i for i in remain_col_ver1]
s

['1participantId',
 '1teamId',
 '1championId',
 '1stats.win',
 '1stats.kills',
 '1stats.deaths',
 '1stats.assists',
 '1stats.longestTimeSpentLiving',
 '1stats.visionScore',
 '1stats.goldEarned',
 '1stats.totalMinionsKilled',
 '1stats.champLevel',
 '1stats.visionWardsBoughtInGame',
 '1timeline.lane',
 '1timeline.creepsPerMinDeltas.0-10',
 '1timeline.creepsPerMinDeltas.10-20',
 '1timeline.xpPerMinDeltas.0-10',
 '1timeline.xpPerMinDeltas.10-20',
 '1timeline.goldPerMinDeltas.0-10',
 '1timeline.goldPerMinDeltas.10-20']

In [106]:
loading_test[s]

,1participantId,1teamId,1championId,1stats.win,1stats.kills,1stats.deaths,1stats.assists,1stats.longestTimeSpentLiving,1stats.visionScore,1stats.goldEarned,1stats.totalMinionsKilled,1stats.champLevel,1stats.visionWardsBoughtInGame,1timeline.lane,1timeline.creepsPerMinDeltas.0-10,1timeline.creepsPerMinDeltas.10-20,1timeline.xpPerMinDeltas.0-10,1timeline.xpPerMinDeltas.10-20,1timeline.goldPerMinDeltas.0-10,1timeline.goldPerMinDeltas.10-20
0,1,100,517,True,9,6,13,468,23,11204,140,14,3,MIDDLE,5.7,4.7,428.2,457.4,326.6,483.8
1,1,100,246,False,4,4,2,925,19,9305,172,14,4,MIDDLE,6.2,9.1,343.1,628.4,298.6,449.4
2,1,100,517,False,3,7,6,394,15,8555,136,14,2,TOP,5.0,6.7,501.6,408.7,279.9,320.9
3,1,100,39,False,4,11,2,123,4,5584,91,9,1,NONE,6.0,NaN,327.2,NaN,272.3,NaN
4,1,100,81,True,1,0,0,0,6,5244,102,8,0,NONE,7.5,NaN,305.7,NaN,300.1,NaN
5,1,100,164,False,10,8,2,666,26,13599,185,16,2,TOP,6.7,6.4,508.3,536.9,355.0,545.9
6,1,100,126,False,1,6,4,445,11,9558,203,14,0,TOP,7.9,9.0,469.7,570.0,273.2,409.6
7,1,100,44,False,1,5,8,418,41,6060,63,12,8,TOP,3.9,2.1,349.7,297.3,224.2,214.3
8,1,100,131,True,8,2,3,477,18,10333,144,13,2,MIDDLE,6.8,6.3,461.9,388.1,329.4,507.5
9,1,100,236,False,1,4,1,588,12,5415,120,8,2,NONE,8.2,NaN,311.8,NaN,280.3,NaN
